In [ ]:
import os
import cv2
import numpy as np
import sys
sys.path.append('../')
from utils.bleb_analysis import find_blebs, analyze_bleb, make_box_plot_with_significance
    
csv = [["filename", "average_area_percent", "average_major_diameter", "average_minor_diameter", "average_circularity", "bleb_count"]]
analysis_parameters = ["area", "major_diameter", "minor_diameter", "circularity", "bleb_count"]
analysis_dict = {key: {"control": [], "100nm": [], "1um": [], "2.5um": [], "5um": [], "10um": []} for key in analysis_parameters}

folder = "../../analysis/blebs/cellmask_pngs/"
threshold = 500
blurring_threshold = 151

for file in os.listdir(folder):
    
    all_blob_properties = []
    whole_cell_properties = []
    gray_frame = cv2.imread(os.path.join(folder, file), 0)
    
    if gray_frame is None:
        continue
    
    # Crop the image
    ht, wd = gray_frame.shape
    gray_frame = gray_frame[ht // 3:-ht//3, wd// 3:-wd//3]
    original_frame = gray_frame.copy()

    outer_frame = gray_frame
    inner_frame = cv2.medianBlur(gray_frame, blurring_threshold)

    outer_contours = find_blebs(outer_frame)
    inner_contours = find_blebs(inner_frame)

    display_image = original_frame.copy()
    display_image2 = np.ones_like(original_frame) * 255

    for cnt in outer_contours:
        area = cv2.contourArea(cnt)
        if area > threshold * 10:
            properties = analyze_bleb(cnt)
            whole_cell_properties.append(properties)

        if area >= threshold:
            cv2.drawContours(display_image, [cnt], 0, (255, 0, 0), 2)
            cv2.drawContours(display_image2, [cnt], -1, (0), thickness=cv2.FILLED)

    for cnt in inner_contours:
        area = cv2.contourArea(cnt)
        if area >= threshold:
            cv2.drawContours(display_image, [cnt], 0, (255, 0, 0), 2)
            cv2.drawContours(display_image2, [cnt], -1, (255), thickness=cv2.FILLED)

    display_image2 = cv2.bitwise_and(original_frame, original_frame, mask=display_image2)
    
    bleb_array = (display_image2 == 0).astype(np.uint8)
    
    kernel = np.ones((5, 5), np.uint8) 
    bleb_array = cv2.erode(bleb_array, kernel, iterations=2)
    bleb_array = cv2.dilate(bleb_array, kernel, iterations=1)

    bleb_contours, _ = cv2.findContours(bleb_array, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for cnt in bleb_contours:
        area = cv2.contourArea(cnt)
        if area > threshold:
            properties = analyze_bleb(cnt)
            all_blob_properties.append(properties)
    
    total_cell_area = 0
    for props in whole_cell_properties:
        total_cell_area += props["area"]

    total_area = 0
    total_circularity = 0
    total_major_diameter = 0
    total_minor_diameter = 0
    num_blobs = len(all_blob_properties)

    for props in all_blob_properties:
        total_area += props["area"]
        total_circularity += props["circularity"]
        total_major_diameter += props["major_diameter"]
        total_minor_diameter += props["minor_diameter"]

    if num_blobs > 0:
        average_area = total_area * 100 / total_cell_area
        average_circularity = total_circularity / num_blobs
        average_major_diameter = total_major_diameter / num_blobs
        average_minor_diameter = total_minor_diameter / num_blobs
        csv.append([file, average_area, average_major_diameter, average_minor_diameter, average_circularity, num_blobs])
        for key in analysis_dict[analysis_parameters[0]].keys():
            if key in file:
                analysis_dict["area"][key].append(average_area)
                analysis_dict["major_diameter"][key].append(average_major_diameter)
                analysis_dict["minor_diameter"][key].append(average_minor_diameter)
                analysis_dict["circularity"][key].append(average_circularity)
                analysis_dict["bleb_count"][key].append(num_blobs)

df_list = [analysis_dict[key] for key in analysis_parameters]
df_names = ["average_area_percent",  "major_diameter", "minor_diameter", "circularity", "counts"]

for i, df in enumerate(df_list):
    make_box_plot_with_significance(df, df_names[i])
 